# Encoder decoder Network
### This notebook depicts the implementation of a encoder decoder network for seqeunce to sequence task. 

In [1]:
from random import randint
import numpy as np 
from keras.utils import to_categorical
from keras.models import Model 
from keras.layers import Input
from keras.layers import LSTM 
from keras.layers import Dense
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')


In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
n_features = 50+1 
# generate a sequence of random integers 
def generate_sequence(length, n_unique):
    return [randint(1, n_unique-1) for _ in range(length)]

In [4]:
a=[1,22,22,88,91]
a=[0] +a[:-1]
print(to_categorical(a, num_classes=100))

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [5]:
# Prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
    x1, x2, y = [],[],[]
    for _ in range(n_samples):

# generate source sequence 
        source = generate_sequence(n_in, cardinality)

# define target sequence 
        target = source[:n_out]
        target.reverse()

# create padded input target sequence 
        target_in = [0] + target[:-1]
        src_encoded = to_categorical([source], num_classes=cardinality)
        tar_encoded = to_categorical([target], num_classes=cardinality)
        tar2_encoded = to_categorical([target_in], num_classes=cardinality)
# store 
        x1.append(src_encoded)
        x2.append(tar2_encoded)
        y.append(tar_encoded)
    
    return np.array(x1), np.array(x2), np.array(y)

In [6]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
    '''
    n_input: The cardiantly of the inpute sequence 
    n_output: The cardinality of the output sequence 
    n_units: The number of cells to create in the encoder and decoder models. 
    '''
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states=[state_h, state_c]

    #define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # define inference encoder 
    encoder_model = Model(encoder_inputs, encoder_states)

    # define inference decoder 
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
    # return all models
    return model, encoder_model, decoder_model

In [7]:
# generate target given source sequence 
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    '''
    infenc: Encoder model used when making a prediction for a new source sequence 
    infdec: Decoder model used when making a prediction for a new source sequence 
    source: Encoded source sequence 
    n_steps: Number of time stpes in the target sequence
    cardinality: The cardinality of the output sequence, e.g. the number of features, words, or characters for each time step 
    '''
    
# encode
    state = infenc.predict(source)

# start of sequence input
    target_seq = np.array([0.0 for _ in range(cardinality)]).reshape(1,1,cardinality)

# collect predictions 
    output = list()

    for t in range(n_steps):
# predict next char 
        yhat, h, c = infdec.predict([target_seq] + state)

# store prediction 
        output.append(yhat[0,0,:])

# update state 
        state = [h,c]

# update target sequence 
        target_seq = yhat 
    return np.array(output)

In [8]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [9]:
# configure problem 
n_features = 50 + 1
n_steps_in = 6 
n_steps_out = 3 

# define model 
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# generate a single source and target sequence 
x1, x2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
print(x1.shape, x2.shape, y.shape)

(100000, 1, 6, 51) (100000, 1, 3, 51) (100000, 1, 3, 51)


In [10]:
x1 = x1.reshape((x1.shape[0], n_steps_in, n_features)) 
x2 = x2.reshape((x2.shape[0], n_steps_out, n_features))
y= y.reshape((y.shape[0], n_steps_out, n_features))

In [11]:
x2.shape

(100000, 3, 51)

In [12]:
# train model 
train.fit([x1,x2], y, epochs=3)

Epoch 1/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 24s 7ms/step - accuracy: 0.5543 - loss: 1.4471
Epoch 2/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.9984 - loss: 0.0175
Epoch 3/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 23s 7ms/step - accuracy: 0.9994 - loss: 0.0049


In [13]:
# evaluate LSTM 
total, correct = 100, 0 
for _ in range(total):
    x1,x2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
    x1 = x1.reshape((x1.shape[0], n_steps_in, n_features)) 
    x2 = x2.reshape((x2.shape[0], n_steps_out, n_features))
    y= y.reshape((y.shape[0], n_steps_out, n_features))
    
    target = predict_sequence(infenc, infdec, x1, n_steps_out, n_features)
    print(f"Target: {one_hot_decode(target)}, GT: {one_hot_decode(y[0])} ")
    if np.array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
        correct +=1
print(f"Accuracy: {float(correct)/float(total)*100}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Target: [31, 1, 11], GT: [31, 1, 11] 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Target: [12, 38, 27], GT: [12, 38, 27] 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Target: [15, 37, 24], GT: [15, 37, 24] 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Target: [8, 29, 27], GT: [8, 29, 27] 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Target: [35, 43, 45], GT: [35, 43, 45] 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━